# CMB-S4 Instrumentation Summer School

## Holography Lab Notebook #1: Data Visualization

In this notebook, we will:
1. Plot your team's beam data.
2. Compare your data to the expected beams.
3. Determine sources of noise in the measurement.

In [4]:
import numpy
import glob
import os

team=input("Enter your team name:")

if os.path.isdir('Data/'+team):
    print("Welcome "+team+"!")
else:
    print("Team name not found. Check spelling or check Data folder.")

Enter your team name: beam_queens


Welcome beam_queens!
